# 2 Thread

In Chapter 1, we have seen how to use Process to accelerate your python program. In this chapter we will see:

1. What threads are
2. How to create threads and wait for them to finish
3. How to use a ThreadPoolExecutor
4. How to avoid race conditions
5. How to use the common tools that Python threading provide

## 2.1 What Is a Python Thread?

Generally speaking, a thread is a separate flow of execution, which means that your program will have many flow of execution happening at the same time. But for most Python 3 implementations **the threads do not actually execute at the same time**.

This is due to the python interpreter uses [GIL(Global Interpreter Lock)](https://realpython.com/python-gil/) to provide a thread safe memory management environment for the underlying C libraries. And **GIL limits only one Python thread can run at a time**.

## 2.2 When to use python thread?

As only one python thread can run at a time, if **tasks that spend much of their time waiting for external events are generally good candidates for threading**. 

If **tasks that require heavy CPU computation and spend little time waiting for external events might not run faster at all after adding multi threading**.

**The above condition is true for code written in Python and running on the standard CPython implementation**. If your threads are written in C they have the ability to release the GIL and run concurrently. If you are running on a different Python interpreter implementation, check with the documentation to see how it handles threads.

**If you are running a standard Python implementation, writing in only Python, and have a CPU-bound problem, you should check out the multiprocessing module instead.**

So Why python still have thread, if they can't run parallely? 
Thread can provide gains in design clarity for your program architecture. Most of the examples you’ll see in this chapter are not necessarily going to run faster because they use threads. Using threading in them helps to make the design cleaner and easier to reason about.

## 2.3 A simple thread

